In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from helpers import load_config

In [ ]:
PREFIX = "/mnt/c/Users/scl38887/Documents/git/pypsa-nz/"
tech_colors = PREFIX + "config/tech_colors.yaml"

In [ ]:
colors = load_config(tech_colors)    

In [ ]:
country = "NZL"

In [ ]:
# Read data https://ourworldindata.org/renewable-energy
re_prod = pd.read_csv('../../data/owid/modern-renewable-prod.csv')

In [ ]:
re_prod = re_prod[re_prod.Code == country]
re_prod = re_prod.dropna(subset=['Code'])
re_prod = re_prod.set_index('Year')
re_prod = re_prod.sort_index()
re_prod = re_prod.dropna(subset=['Code'])
re_prod = re_prod.rename(columns={'Electricity from wind - TWh': 'wind',
                                  'Electricity from solar - TWh': 'solar',
                                  "Electricity from hydro - TWh": "hydro",
                                  "Other renewables including bioenergy - TWh": "others",})

### Add Growth

In [ ]:
g = 1.4/100 # in %
year=2025 #np.array(range(re_prod.index[-1]+1,2050+1,1))
demand=10
floor=7
solar_growth =  (demand * g) * (1+g) ** (year-2023)
#solar_growth_df = pd.DataFrame(solar_growth, index=year, columns=["solar"])

In [ ]:
solar_growth

### Plot

In [ ]:
# Set up figure
fig = plt.figure(facecolor='white', figsize=(10, 5))

ax = fig.add_subplot(1,1,1)

ax.plot("wind", data=re_prod, color=colors["tech_colors"]["onshore wind"], label="Wind")
ax.plot("solar", data=re_prod, color=colors["tech_colors"]["solar PV"], label="Solar")

# Limit the plot
ax.set_xlim(1990, 2045)
# ax.set_ylim(0,100)

# Further elements
ax.set_ylabel("Electricity generation in TWh")
ax.set_xlabel("Year")
plt.grid(alpha=0.4)

# Create graph and save it
#fig.savefig(snakemake.output.reshares, bbox_inches="tight")

plt.show()